In [4]:
import os
import keras, tensorflow
from keras.preprocessing import image
import cv2
import numpy as np
import pandas as pd


In [5]:
data = []
categories = [ 'incorrect_mask','with_mask' , 'without_mask']
for i in categories: 
    path = os.path.join('train' , i )
    label = categories.index(i) 
    for file in os.listdir(path):
        #print(file)
        img_path = os.path.join(path , file)
        #print(img_path)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append([img,label])       
       

In [3]:

# path = os.path.join('train' , 'with_mask' )
# img_path = os.path.join(path , '0-with-mask.jpg')
# print(img_path)
# img = cv2.imread(img_path)
# img = cv2.resize(img,(224,224))
# cv2.imshow("image",img)
# cv2.waitKey(10000)
# cv2.destroyAllWindows()

In [6]:
len(data)

2079

In [7]:
import random
random.shuffle(data)

In [8]:
x= [] 
y = []
for features,label in data:
    x.append(features)
    y.append(label)

In [9]:
len(x)
len(y)

2079

In [10]:
x = np.array(x)
y = np.array(y)

In [11]:
y

array([0, 0, 0, ..., 1, 1, 0])

In [12]:
print(x.shape, y.shape)

(2079, 224, 224, 3) (2079,)


In [13]:
from tensorflow.keras.utils import to_categorical
y= to_categorical(y)

In [14]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [15]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split( x , y , random_state= 0 , test_size= 0.2)

In [16]:
x_train.shape


(1663, 224, 224, 3)

In [17]:
y_train.shape

(1663, 3)

In [18]:
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.4, 1.5],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented data from the training array
training_set = train_datagen.flow(x_train, y_train,
                                  batch_size=32,
                                  shuffle=True)

# Generate batches of augmented data from the testing array
test_set = test_datagen.flow(x_test, y_test,
                             batch_size=32,
                             shuffle=False)


In [21]:

from keras.applications.vgg16 import VGG16
vgg =VGG16()
vgg = VGG16(weights='imagenet', include_top=False,input_shape = (224,224,3))
#vgg.summary()

58889256/58889256 [==============================] - 20s 0us/step


In [19]:
layer = vgg.layers[-1]
print(layer.output_shape)  

(None, 1000)


In [22]:
from keras import Sequential
model = Sequential()

for layer in vgg.layers[: -1]:
    model.add(layer)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [37]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [23]:
from keras.layers import Dropout
dropout1 = Dropout(0.1)
dropout2 = Dropout(0.1)

In [24]:
fc1 = model.layers[-3]
fc2 = model.layers[-2]
predictions = model.layers[-1]

In [25]:
from keras.models import Model
x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = predictions(x)

# Create a new model
model2 = Model(inputs=model.input , outputs=predictors)

In [26]:
#freezing the parametre
for layer in model2.layers:
    layer.trainable = False
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [39]:
from keras.layers import Dense
final_model = Sequential()
final_model.add(model2)
final_model.add(Flatten())
final_model.add(Dense(3, activation='softmax'))

#model2.add(Dense(1, activation='sigmoid'))
final_model.summary()






Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 14, 14, 512)       14714688  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 3)                 301059    
                                                                 
Total params: 15,015,747
Trainable params: 301,059
Non-trainable params: 14,714,688
_________________________________________________________________


In [27]:
tensorflow.test.is_gpu_available()
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
checkpoint = ModelCheckpoint("vgg16.h5",monitor = "val_accuracy",save_best_only = True,verbose=1)
earlystop = EarlyStopping(monitor="val_accuracy",patience=8,verbose=1)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [40]:

final_model.compile( optimizer= 'Adam' , loss ='categorical_crossentropy' , metrics= ['accuracy'])
#metrics=[ metrics.MeanSquaredError(), metrics.AUC()] 
#final_model.fit(x_train , y_train , epochs= 5 , validation_data=( x_test , y_test))
batch_size = 32  # Specify the desired batch size


History=final_model.fit(training_set, epochs=15, callbacks=[checkpoint,earlystop],validation_data=(test_set))


Epoch 1/15
52/52 [==============================] - ETA: 0s - loss: 1.1735 - accuracy: 0.7426
Epoch 1: val_accuracy improved from -inf to 0.95433, saving model to vgg16.h5
52/52 [==============================] - 341s 7s/step - loss: 1.1735 - accuracy: 0.7426 - val_loss: 0.1484 - val_accuracy: 0.9543
Epoch 2/15
52/52 [==============================] - ETA: 0s - loss: 0.5533 - accuracy: 0.8310
Epoch 2: val_accuracy did not improve from 0.95433
52/52 [==============================] - 397s 8s/step - loss: 0.5533 - accuracy: 0.8310 - val_loss: 0.2470 - val_accuracy: 0.9495
Epoch 3/15
52/52 [==============================] - ETA: 0s - loss: 0.3485 - accuracy: 0.8966
Epoch 3: val_accuracy improved from 0.95433 to 0.96635, saving model to vgg16.h5
52/52 [==============================] - 348s 7s/step - loss: 0.3485 - accuracy: 0.8966 - val_loss: 0.1147 - val_accuracy: 0.9663
Epoch 4/15
52/52 [==============================] - ETA: 0s - loss: 0.2780 - accuracy: 0.9092
Epoch 4: val_accuracy im

In [52]:

import cv2
img = cv2.imread(r"C:\Users\Gyaneshwar\Desktop\Project 2\test2.PNG")
if img is not None:
    # Resize the image
    resized_img = cv2.resize(img, (224, 224))

    # Do something with the resized image
    # ...

else:
    print("Failed to load the image")




In [53]:
y_pred = final_model.predict(resized_img.reshape(1,224,224,3))


1/1 [==============================] - 0s 150ms/step


In [54]:
print(y_pred[0][0])
print(y_pred[0][1])
print(y_pred[0][2])


1.0
0.0
0.0


In [50]:
def detect_face_mask(img):
     y_pred = model.predict(img.reshape(1,224,224,3))
    # max_index = y_pred.index(max(y_pred[0]))
     return y_pred
     


def draw_label(img ,text, pos, bg_color):
    text_size =cv2.getTextSize(text , cv2.FONT_HERSHEY_COMPLEX,1 , cv2.FILLED)

    end_x = pos[0] + text_size[0][1] + 2
    end_y = pos[1] + text_size[0][1] - 2
    cv2.rectangle(img,pos,(end_x , end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_COMPLEX , 1, (0,0,0),1,cv2.LINE_AA)

In [45]:
cap = cv2.VideoCapture(0)

In [58]:
while True:
      ret , frame  = cap.read()
      
      img = cv2.resize(frame,(224,224))
      y_pred = detect_face_mask(img)
      print(y_pred[0][2])
      #if(y_pred[0][2]>0.5):
       #     draw_label(frame , "NO MASK" , (30,30) ,(0,0,0))
      #if(y_pred[0][0]>0.51):
      #      draw_label(frame , "NOT PROPER MASK" , (30,30) ,(0,0,0))
      #else:
       #     draw_label(frame , "MASK" , (30,30) ,(0,0,0))

      coods  = face_sqaure(frame)
      for x,y,w,h in coods:
        cv2.rectangle(frame, (x,y),(x+w , y+h),(255,0,0) , 2)
      cv2.imshow("Windows", frame)
      
      
      cv2.imshow("window" , frame)
      if cv2.waitKey(1) & 0xFF == ord('x'):
         break
cv2.destroyAllWindows()

1/1 [==============================] - 0s 157ms/step
[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 ...
 [ 0.         0.         0.        ...  3.8900206  4.52857   44.820084 ]
 [ 0.         0.         0.        ...  0.         5.270244   1.3614038]
 [ 0.         0.         0.        ...  0.        43.368652   0.       ]]
1/1 [==============================] - 0s 151ms/step
[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 ...
 [ 0.         0.         0.        ...  4.850369   0.        38.97506  ]
 [ 0.         0.         0.        ...  0.         0.         1.7913667]
 [ 0.         0.         0.        ...  0.        39.811035   0.       ]]
1/1 [=

coods  = face_sqaure(frame)
      for x,y,w,h in coods:
        cv2.rectangle(frame, (x,y),(x+w , y+h),(255,0,0) , 2)
      cv2.imshow("Windows", frame)
      
      
      cv2.imshow("window" , frame)
      if cv2.waitKey(1) & 0xFF == ord('x'):
         break
cv2.destroyAllWindows()



def draw_label(img ,text, pos, bg_color):
    text_size =cv2.getTextSize(text , cv2.FONT_HERSHEY_COMPLEX,1 , cv2.FILLED)

    end_x = pos[0] + text_size[0][1] + 2
    end_y = pos[1] + text_size[0][1] - 2
    cv2.rectangle(img,pos,(end_x , end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_COMPLEX , 1, (0,0,0),1,cv2.LINE_AA)

In [47]:
haar = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def face_sqaure(img):
    faces =[]
    #img = cv2.resize(img,(224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = haar.detectMultiScale(img, 1.3, 5)
    
    #faces = haar.detectMultiScale(img, 1.0485258, 6)
    #coods = haar.detectMultiScale(img) 
    
    return faces

    

In [99]:
while True:
    ret, frame = cap.read()
    coods=[]
    coods  = face_sqaure(frame)
    for x,y,w,h in coods:
        cv2.rectangle(frame, (x,y),(x+w , y+h),(255,0,0) , 2)
    cv2.imshow("Windows", frame)
    if cv2.waitKey(1)  & 0xFF == ord('x'):
        break
cv2.destroyAllWindows()
